In [3]:
%load_ext autoreload
%autoreload 2
from datasets import Dataset, Image, load_dataset, load_from_disk
import os
from transformers import pipeline
import tqdm
import numpy as np
import sys
sys.path.append("..")
from utils.data_utils import make_hf_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dataset = make_hf_dataset(input_folder='../../data/landsat/inputs', target_folder='../../data/landsat/targets',
                          masks_folder='../../data/landsat/masks',
                           save_to_disk=True, dset_location='../../data/landsat/dset.hf')

Saving dset to disk...


Saving the dataset (1/1 shards): 100%|██████████| 9076/9076 [01:15<00:00, 120.83 examples/s]


In [4]:
dataset = load_from_disk('../../data/landsat/dset.hf')

In [5]:
dataset.push_to_hub(f"Kornberg/landsat_captions", token=os.environ['HF_ACCESS_TOKEN'])

Uploading the dataset shards: 100%|██████████| 1/1 [04:10<00:00, 250.69s/it]
